# Moral Machine Contrastive Pairs Dataset

### Step 1: Setup

## Imports

In [ ]:
# Import necessary libraries
# colab
from google.colab import drive, userdata
# os
import os
# data handlers
import pandas as pd
import numpy as np


In [ ]:
# --- 1. Mount Google Drive ---
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Drive mounted successfully.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted successfully.


In [ ]:
# Retrieve the secret value for project path
WORKING_DIR = userdata.get('moral_path')

# change working directory
os.chdir(WORKING_DIR)
# check the current directory
!pwd

/content/drive/My Drive/_PhD/Moral-Reasoning/Experiments/Data


If unzipping raw data - else skip

## Data Setup

Config

In [ ]:
# --- Configuration ---

# 1. Define the path to the source file
SOURCE_FILE_PATH = './data/original/SharedResponses.csv'

# 2. Define the output directory for new datasets.
OUTPUT_DIR = './data/processed/contrastive_datasets'

# 3. Define all columns that represent a character in the scene.
CHARACTER_COLS = [
    'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman',
    'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal',
    'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete',
    'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat'
]

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Libraries imported. Ready to load data from: {SOURCE_FILE_PATH}")
print(f"Output datasets will be saved to: {OUTPUT_DIR}")

Libraries imported. Ready to load data from: ./moral_machine/SharedResponses.csv
Output datasets will be saved to: ./moral_machine/contrastive_datasets


Read in Data

In [ ]:
# Read header row (nrows=0) and get the column names
try:
    column_names = pd.read_csv(SOURCE_FILE_PATH, nrows=0).columns.tolist()
    print("Column names in the file:")
    print(column_names)
except FileNotFoundError:
    print(f"Error:'{SOURCE_FILE_PATH}' not in workspace.")

Here are the actual column names in the file:
['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder', 'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown', 'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']


In [ ]:
# --- 5. Read the CSV into a DataFrame ---
columns_to_load = [
    'ResponseID',
    'ExtendedSessionID',
    'UserID',
    'ScenarioOrder',
    'Intervention',
    'PedPed',
    'Barrier',
    'CrossingSignal',
    'AttributeLevel',
    'ScenarioTypeStrict',
    'ScenarioType',
    'DefaultChoice',
    'NonDefaultChoice',
    'DefaultChoiceIsOmission',
    'NumberOfCharacters',
    'DiffNumberOFCharacters',
    'Saved', 'Template',
    'DescriptionShown',
    'LeftHand',
    'UserCountry3',
    'Man',
    'Woman',
    'Pregnant',
    'Stroller',
    'OldMan',
    'OldWoman',
    'Boy',
    'Girl',
    'Homeless',
    'LargeWoman',
    'LargeMan',
    'Criminal',
    'MaleExecutive',
    'FemaleExecutive',
    'FemaleAthlete',
    'MaleAthlete',
    'FemaleDoctor',
    'MaleDoctor',
    'Dog',
    'Cat'
  ]

print(f"\nLoading '{SOURCE_FILE_PATH}' into DataFrame...")
try:
    df= pd.read_csv(SOURCE_FILE_PATH, usecols=columns_to_load, nrows=100000)
    # fill any missing data with 0s
    df[CHARACTER_COLS] = df[CHARACTER_COLS].fillna(0)
    print("\n✅ Successfully loaded! Here are the first 5 rows:")
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")


Loading './moral_machine/SharedResponses.csv' into DataFrame...

✅ Successfully loaded! Here are the first 5 rows:
          ResponseID              ExtendedSessionID        UserID  \
0  2222bRQqBTZ6dLnPH    32757157_6999801415950060.0  6.999801e+15   
1  2222sJk4DcoqXXi98        1043988516_3525281295.0  3.525281e+09   
2  2223CNmvTr2Coj4wp  -1613944085_422160228641876.0  4.221602e+14   
3  2223Xu54ufgjcyMR3   1425316635_327833569077076.0  3.278336e+14   
4  2223jMWDEGNeszivb  -1683127088_785070916172117.0  7.850709e+14   

   ScenarioOrder  Intervention  PedPed  Barrier  CrossingSignal  \
0              7             0       0        0               1   
1              2             0       0        0               0   
2             10             0       1        0               1   
3             11             0       0        1               0   
4              8             0       1        0               2   

  AttributeLevel ScenarioTypeStrict  ... LargeMan Criminal MaleExe

## Scenario Parsing and Text Templating

In [ ]:
def describe_group(chars_dict, status_text, is_passenger):
    """
    Converts a dictionary of characters and a status into a readable string.
    Example: {'Man': 1, 'Girl': 2}, "crossing illegally" -> "1 Man and 2 Girls who are crossing illegally"
    """
    if is_passenger:
        # Passengers don't have a crossing status
        count = sum(chars_dict.values())
        plural = "Passenger" if count == 1 else "Passengers"
        return f"{count} {plural}"

    # Build the character list
    char_list = []
    for char, count in chars_dict.items():
        if count > 0:
            # Handle plural names (e.g., "Woman" -> "Women", "Dog" -> "Dogs")
            if char == 'Woman':
                name = 'Woman' if count == 1 else 'Women'
            elif char == 'OldWoman':
                name = 'Old Woman' if count == 1 else 'Old Women'
            elif char == 'LargeWoman':
                name = 'Large Woman' if count == 1 else 'Large Women'
            elif char == 'Man':
                name = 'Man' if count == 1 else 'Men'
            elif char == 'OldMan':
                name = 'Old Man' if count == 1 else 'Old Men'
            elif char == 'LargeMan':
                name = 'Large Man' if count == 1 else 'Large Men'
            elif char == 'Boy':
                name = 'Boy' if count == 1 else 'Boys'
            elif char == 'Girl':
                name = 'Girl' if count == 1 else 'Girls'
            elif char == 'Dog':
                name = 'Dog' if count == 1 else 'Dogs'
            elif char == 'Cat':
                name = 'Cat' if count == 1 else 'Cats'
            else:
                # For 'Criminal', 'Pregnant', 'Stroller', 'Executive', 'Athlete', 'Doctor', 'Homeless'
                name = char if count == 1 else f"{char}s"

            char_list.append(f"{count} {name}")

    # Join the list with commas and 'and'
    if not char_list:
        return f"an empty lane {status_text}"
    elif len(char_list) == 1:
        desc = char_list[0]
    else:
        desc = ", ".join(char_list[:-1]) + " and " + char_list[-1]

    return f"{desc} {status_text}"

def parse_stay_scenario(row):
    """
    Parses the "Stay the course" group, described
    by the main character and crossing signal columns.
    """
    scenario = {
        'chars': {},
        'total_count': 0,
        'status_text': '',
        'crossing_signal': row['CrossingSignal'],
        'has_criminals': False,
        'is_passenger': False
    }

    for char in CHARACTER_COLS:
        count = int(row[char])
        if count > 0:
            scenario['chars'][char] = count
            scenario['total_count'] += count
            if char == 'Criminal':
                scenario['has_criminals'] = True

    # Define legal crossing
    if row['CrossingSignal'] == 1:
        scenario['status_text'] = "who are crossing legally"
    elif row['CrossingSignal'] == -1:
        scenario['status_text'] = "who are crossing illegally"
    else: # 0 or NaN
        scenario['status_text'] = "" # No status

    return scenario

def parse_swerve_scenario(row, stay_total_count):
    """
    Parses the "Swerve to avoid" group by contrasting
    it with the "Stay the course" group.
    """
    scenario = {
        'chars': {},
        'total_count': 0,
        'status_text': '',
        'crossing_signal': 0, # Default
        'has_criminals': False,
        'is_passenger': False
    }

    # IF the swerve group is Passenger
    # Assumption: We don't know who the passengers are, so we use a generic "Passenger"
    # Also assumes passengers don't have a crossing signal (check legality) or criminal status
    if row['Barrier'] == 1:
        scenario['is_passenger'] = True
        scenario['total_count'] = stay_total_count - row['DiffNumberOFCharacters']
        scenario['chars']['Passenger'] = scenario['total_count']

    # The swerve group is other Pedestrians
    else:
        scenario['is_passenger'] = False
        scenario['total_count'] = stay_total_count - row['DiffNumberOFCharacters']

        # DEDUCTIVE METHODS FOR DEFINING GROUP ATTRIBUTES

        # 1. Deducing legality of crossing
        # If the attribute being tested is legality, the swerve group is the opposite
        if row['AttributeLevel'] == 'CrossingSignal':
            scenario['crossing_signal'] = -row['CrossingSignal'] # Opposite of stay group
        else:
            scenario['crossing_signal'] = row['CrossingSignal'] # Same as stay group

        # 2. Deduce criminality of person
        # If the attribute is 'Criminals', the swerve group is the opposite
        if row['AttributeLevel'] == 'ScenarioType' and 'Criminals' in row['ScenarioType']:
            scenario['has_criminals'] = not row['Criminal'] > 0

        # 3. Add generic characters to match the total count
        # We only know legality, criminality of person
        # Defined as "Pedestrian" text
        if scenario['total_count'] > 0:
             scenario['chars']['Pedestrian'] = scenario['total_count']

        # 4. Set status text based on deduced legality
        if scenario['crossing_signal'] == 1:
            scenario['status_text'] = "who are crossing legally"
        elif scenario['crossing_signal'] == -1:
            scenario['status_text'] = "who are crossing illegally"
        else:
            scenario['status_text'] = ""

    return scenario

def generate_text_pair(row, stay_scenario, swerve_scenario):
    """
    Applies our chosen template to the two parsed scenarios.
    """
    # Starter for all senarios
    context = "A self-driving car is in an unavoidable crash scenario."

    # --- Generate Text for STAY THE COURSE (innaction) ---
    stay_description = describe_group(
        stay_scenario['chars'],
        stay_scenario['status_text'],
        stay_scenario['is_passenger']
    )
    text_stay = f"{context} The car stays on its course. This action results in the deaths of {stay_description}."

    # --- Generate Text for SWERVE (action) ---
    swerve_description = describe_group(
        swerve_scenario['chars'],
        swerve_scenario['status_text'],
        swerve_scenario['is_passenger']
    )
    text_swerve = f"{context} The car swerves. This action results in the deaths of {swerve_description}."

    return text_stay, text_swerve

print("Helper functions defined for parsing and text generation.")

Helper functions defined for parsing and text generation.


## Defining the Ethical Principles

In [ ]:
def get_utilitarian_choice(stay_info, swerve_info):
    """
    Applies the Utilitarian rule: save the most lives.
    Returns 'stay', 'swerve', or 'equal'.
    """
    stay_deaths = stay_info['total_count']
    swerve_deaths = swerve_info['total_count']

    if stay_deaths < swerve_deaths:
        return 'stay'
    elif swerve_deaths < stay_deaths:
        return 'swerve'
    else:
        return 'equal'

def get_deontological_choice(row, stay_info, swerve_info):
    """
    Applies the hierarchical Deontological rules:
    1. Anti-Sacrifice (Don't kill passengers)
    2. Rule of Law (Spare legal pedestrians)
    3. Innocence (Spare non-criminals)
    4. Inaction (Default to 'stay')
    """

    # Rule 1: Anti-Sacrifice (Barrier)
    # If swerving hits passengers (Barrier=1), the D choice is ALWAYS 'stay'.
    if row['Barrier'] == 1:
        return 'stay'

    # Rule 2: Rule of Law (Legality)
    # This rule applies only if one group is legal (1) and the other is illegal (-1).
    stay_legal = stay_info['crossing_signal']
    swerve_legal = swerve_info['crossing_signal']

    if stay_legal == 1 and swerve_legal == -1:
        return 'stay'  # Spare the legal group
    if swerve_legal == 1 and stay_legal == -1:
        return 'swerve' # Spare the legal group

    # Rule 3: Principle of Innocence (Criminals)
    # This rule applies if one group has criminals and the other does not.
    stay_has_criminals = stay_info['has_criminals']
    swerve_has_criminals = swerve_info['has_criminals']

    if not stay_has_criminals and swerve_has_criminals:
        return 'stay' # Spare the innocent group
    if not swerve_has_criminals and stay_has_criminals:
        return 'swerve' # Spare the innocent group

    # Rule 4: Inaction (Omission)
    # If no other rules apply, the default D choice is inaction ('stay').
    return 'stay'

print("Ethical principle functions defined (Utilitarian and Deontological).")

Ethical principle functions defined (Utilitarian and Deontological).


## Main Processing Loop

In [ ]:
print("Starting main processing loop...")

# Create a list to hold all processed data
processed_data = []

# Iterate over every row in the DataFrame
# itertuples() aster iteration than .iterrows()
# for row in df.iterrows(): # comment out
for row in df.itertuples():

    # Step 1: Parse the two scenarios
    # Pass the row (tuple) from dataframe as a dictionary-like object
    # use the original dataframe by index
    # Helper functions expect column names.
    row_data = df.iloc[row.Index]
    # print(row_data)

    stay_scenario = parse_stay_scenario(row_data)
    swerve_scenario = parse_swerve_scenario(row_data, stay_scenario['total_count'])

    # Step 2: Generate the text for each choice
    text_stay, text_swerve = generate_text_pair(row_data, stay_scenario, swerve_scenario)

    # Step 3: Classify the choices
    utilitarian_choice = get_utilitarian_choice(stay_scenario, swerve_scenario)
    deontological_choice = get_deontological_choice(row_data, stay_scenario, swerve_scenario)

    # Step 4: Store the results
    processed_data.append({
        'ResponseID': row.ResponseID,
        'text_stay': text_stay,
        'text_swerve': text_swerve,
        'utilitarian_choice': utilitarian_choice,
        'deontological_choice': deontological_choice
    })

# Convert the list of results into a new DataFrame
processed_df = pd.DataFrame(processed_data)

print(f"\nProcessing complete. Processed {len(processed_df)} scenarios.")
print("--- Processed Data Head ---")
display(processed_df.head())

Starting main processing loop...
This may take a few minutes depending on the dataset size.

Processing complete. Processed 100000 scenarios.
--- Processed Data Head ---


,ResponseID,text_stay,text_swerve,utilitarian_choice,deontological_choice
0,2222bRQqBTZ6dLnPH,A self-driving car is in an unavoidable crash ...,A self-driving car is in an unavoidable crash ...,equal,stay
1,2222sJk4DcoqXXi98,A self-driving car is in an unavoidable crash ...,A self-driving car is in an unavoidable crash ...,equal,stay
2,2223CNmvTr2Coj4wp,A self-driving car is in an unavoidable crash ...,A self-driving car is in an unavoidable crash ...,equal,stay
3,2223Xu54ufgjcyMR3,A self-driving car is in an unavoidable crash ...,A self-driving car is in an unavoidable crash ...,equal,stay
4,2223jMWDEGNeszivb,A self-driving car is in an unavoidable crash ...,A self-driving car is in an unavoidable crash ...,swerve,swerve


## Creating the Contrastive Pair Datasets

In [ ]:
print("Building contrastive pair datasets...")

# --- Dataset 1: Utilitarian vs. Non-Utilitarian ---
# This dataset trains the model to be purely utilitarian.
# We take all scenarios where the choice is not 'equal'.
df_u_vs_non_u = processed_df[processed_df['utilitarian_choice'] != 'equal'].copy()

# 'chosen' is the utilitarian text, 'rejected' is the non-utilitarian text
df_u_vs_non_u['chosen'] = np.where(
    df_u_vs_non_u['utilitarian_choice'] == 'stay',
    df_u_vs_non_u['text_stay'],
    df_u_vs_non_u['text_swerve']
)
df_u_vs_non_u['rejected'] = np.where(
    df_u_vs_non_u['utilitarian_choice'] == 'stay',
    df_u_vs_non_u['text_swerve'],
    df_u_vs_non_u['text_stay']
)

# Save the final dataset
dataset1 = df_u_vs_non_u[['chosen', 'rejected']]
path1 = os.path.join(OUTPUT_DIR, 'utilitarian_vs_non_utilitarian.csv')
dataset1.to_csv(path1, index=False)
print(f"1. Saved 'Utilitarian vs. Non-Utilitarian' dataset to {path1} ({len(dataset1)} pairs)")


# --- Dataset 2: Deontological vs. Non-Deontological ---
# This dataset trains the model to be purely deontological.
# We can use all scenarios, as 'stay' is always the fallback D choice.
df_d_vs_non_d = processed_df.copy() # No filter needed

# 'chosen' is the deontological text, 'rejected' is the non-deontological text
df_d_vs_non_d['chosen'] = np.where(
    df_d_vs_non_d['deontological_choice'] == 'stay',
    df_d_vs_non_d['text_stay'],
    df_d_vs_non_d['text_swerve']
)
df_d_vs_non_d['rejected'] = np.where(
    df_d_vs_non_d['deontological_choice'] == 'stay',
    df_d_vs_non_d['text_swerve'],
    df_d_vs_non_d['text_stay']
)

# Save the final dataset
dataset2 = df_d_vs_non_d[['chosen', 'rejected']]
path2 = os.path.join(OUTPUT_DIR, 'deontological_vs_non_deontological.csv')
dataset2.to_csv(path2, index=False)
print(f"2. Saved 'Deontological vs. Non-Deontological' dataset to {path2} ({len(dataset2)} pairs)")


# --- Dataset 3: Utilitarian vs. Deontological (Conflict Scenarios) ---
# This dataset contains ONLY the scenarios where the U and D rules conflict
# This is for training the model on the hard trade-offs
df_conflict = processed_df[
    (processed_df['utilitarian_choice'] != 'equal') &
    (processed_df['utilitarian_choice'] != processed_df['deontological_choice'])
].copy()

# Dataset framed as 'utilitarian' (chosen) vs. 'deontological' (rejected)
# Can be swapped to to train the model to prefer deontology
df_conflict['chosen'] = np.where(
    df_conflict['utilitarian_choice'] == 'stay',
    df_conflict['text_stay'],
    df_conflict['text_swerve']
)
df_conflict['rejected'] = np.where(
    df_conflict['deontological_choice'] == 'stay',
    df_conflict['text_stay'],
    df_conflict['text_swerve']
)

# Save the final dataset
dataset3 = df_conflict[['chosen', 'rejected']]
path3 = os.path.join(OUTPUT_DIR, 'utilitarian_vs_deontological_conflict.csv')
dataset3.to_csv(path3, index=False)
print(f"3. Saved 'Utilitarian vs. Deontological' conflict dataset to {path3} ({len(dataset3)} pairs)")

print("\n--- All tasks complete. ---")

Building contrastive pair datasets...
1. Saved 'Utilitarian vs. Non-Utilitarian' dataset to ./moral_machine/contrastive_datasets/utilitarian_vs_non_utilitarian.csv (24124 pairs)
2. Saved 'Deontological vs. Non-Deontological' dataset to ./moral_machine/contrastive_datasets/deontological_vs_non_deontological.csv (100000 pairs)
3. Saved 'Utilitarian vs. Deontological' conflict dataset to ./moral_machine/contrastive_datasets/utilitarian_vs_deontological_conflict.csv (21109 pairs)

--- All tasks complete. ---


## Check New Dataset Created Correctly

In [ ]:
print("--- 'utilitarian_vs_deontological_conflict.csv' ---")

try:
    example_df = pd.read_csv(path3)

    # Set display options to show full text
    pd.set_option('display.max_colwidth', None)

    display(example_df.head(5))

except FileNotFoundError:
    print(f"Could not load example file at {path3}")
except Exception as e:
    print(f"An error occurred: {e}")

--- Example from 'utilitarian_vs_deontological_conflict.csv' ---


,chosen,rejected
0,A self-driving car is in an unavoidable crash scenario. The car swerves. This action results in the deaths of 3.0 Passengers.,"A self-driving car is in an unavoidable crash scenario. The car stays on its course. This action results in the deaths of 1 Stroller, 1 FemaleAthlete, 1 FemaleDoctor and 1 Cat ."
1,A self-driving car is in an unavoidable crash scenario. The car swerves. This action results in the deaths of 1.0 Passenger.,"A self-driving car is in an unavoidable crash scenario. The car stays on its course. This action results in the deaths of 1 Old Woman, 1 Homeless, 1 MaleAthlete, 1 Dog and 1 Cat ."
2,A self-driving car is in an unavoidable crash scenario. The car swerves. This action results in the deaths of an empty lane .,A self-driving car is in an unavoidable crash scenario. The car stays on its course. This action results in the deaths of 1 Homeless and 1 MaleDoctor .
3,A self-driving car is in an unavoidable crash scenario. The car swerves. This action results in the deaths of 1.0 Pedestrian .,"A self-driving car is in an unavoidable crash scenario. The car stays on its course. This action results in the deaths of 1 Stroller, 1 MaleExecutive and 1 MaleAthlete ."
4,A self-driving car is in an unavoidable crash scenario. The car swerves. This action results in the deaths of 1.0 Pedestrian .,"A self-driving car is in an unavoidable crash scenario. The car stays on its course. This action results in the deaths of 1 Pregnant, 1 Boy, 1 Girl, 1 Homeless and 1 MaleAthlete ."
